<a href="https://colab.research.google.com/github/Jayesh-2003/Academic-Bot---PEFT/blob/main/Colab-Hosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install huggingface_hub langchain_groq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/5

In [ ]:
!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=3510601 sha256=8fe946c097cb3aadf11d71537b421d3ae2db6b0363963beb8416328dbbd06f09
  Stored in directory: /root/.cache/pip/wheels/f8/b0/a2/f47d952aec7ab061b9e2a345e23a1e1e137beb7891259e3d0c
Successfully built llama-cpp-python


In [ ]:
!python3

Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 

In [ ]:
# app.py
%%writefile app.py
from llama_cpp import Llama
import streamlit as st
from langchain_groq import ChatGroq

# Set your GROQ API key (replace with your actual API key)
groq_api_key = "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"


llama_model=Llama.from_pretrained(
	repo_id="cryotron/chatbot_academic_3rd_Year_GUFF_Quantized4Bit",
	filename="unsloth.Q4_K_M.gguf",
)


# Initialize the GROQ model
groq_model = ChatGroq(groq_api_key=groq_api_key, model_name="llama3-groq-70b-8192-tool-use-preview")

# Define the Alpaca prompt template
alpaca_prompt = """You are a Chatbot for engineering students answering their questions.
### Instruction:
{}

### Input:
{}

### Response:
{}
"""

# Function to generate text using the llama-cpp model
def generate_text(model, prompt, max_new_tokens=1024):
    response = model(prompt, max_tokens=max_new_tokens, stop=["###"])
    return response['choices'][0]['text']

# Function to route question to both LLM and GROQ
def route_question(question):
    llm_prompt = alpaca_prompt.format(question, "", "")
    llm_response = generate_text(llama_model, llm_prompt)

    groq_question = f"This is a Computer Engineering Syllabus Question: {question} Give answer"
    groq_response = groq_model.invoke(groq_question)

    return {
        "llm_response": llm_response.strip(),
        "groq_response": groq_response
    }

# Streamlit App Layout
st.title("Engineering Chatbot")
st.write("This app allows you to ask questions and get responses from both a Local LLM and GROQ.")

user_question = st.text_input("Ask a question related to Computer Engineering:")

if st.button("Get Answer"):
    if user_question.strip():
        responses = route_question(user_question)

        st.subheader("LLM Response:")
        st.write(responses["llm_response"])

        st.subheader("GROQ Response:")
        if hasattr(responses["groq_response"], 'content'):
            st.write(responses["groq_response"].content)
        else:
            st.write(responses["groq_response"])
    else:
        st.write("Please ask a valid question.")


Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.185.137.62:8501

Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) y
your url is: https://short-phones-tan.loca.lt
unsloth.Q4_K_M.gguf: 100% 4.92G/4.92G [02:46<00:00, 29.6MB/s]
llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--cryotron--q4_k_m_academic/snapshots/fc0f2e9d22f17c663de9c740762c7ae4a15328ad/./unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llam

In [ ]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="cryotron/16bit_academic",
	filename="unsloth.F16.gguf",
)



unsloth.F16.gguf:   0%|          | 0.00/16.1G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--cryotron--16bit_academic/snapshots/12c158bc23ae5cc78459625abc54f53201ab0841/./unsloth.F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3 8b Bnb 4bit
llama_model_loader: - kv   3:                       general.organization str              = Unsloth
llama_model_loader: - kv   4:                           general.finetune str              = bnb-4bit
llama_model_loader: - kv   5:                           general.basename str              = llama-3
llama_model_loader: - kv   6:                       

In [ ]:
from llama_cpp import Llama
import streamlit as st
from langchain_groq import ChatGroq

# Set your GROQ API key (replace with your actual API key)
groq_api_key = "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"



# Initialize the GROQ model
groq_model = ChatGroq(groq_api_key=groq_api_key, model_name="llama3-groq-70b-8192-tool-use-preview")

# Define the Alpaca prompt template
alpaca_prompt = """You are a Chatbot for engineering students answering their questions.
### Instruction:
{}

### Input:
{}

### Response:
{}
"""

# Function to generate text using the llama-cpp model
def generate_text(model, prompt, max_new_tokens=1024):
    response = model(prompt, max_tokens=max_new_tokens, stop=["###"])
    return response['choices'][0]['text']

# Function to route question to both LLM and GROQ
def route_question(question):
    llm_prompt = alpaca_prompt.format(question, "", "")
    llm_response = generate_text(llm, llm_prompt)

    groq_question = f"This is a Computer Engineering Syllabus Question: {question} Give answer"
    groq_response = groq_model.invoke(groq_question)

    return {
        "llm_response": llm_response.strip(),
        "groq_response": groq_response
    }


In [ ]:
a=route_question("What is extension and intension?")

Llama.generate: 28 prefix-match hit, remaining 1 prompt tokens to eval
